In [54]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

class Args:
    def __init__(self):
        self.batch_size = 100
        self.epochs = 100
        self.no_cuda = False
        self.cuda = False
        self.seed = 1
        self.log_interval = 10
        
args = Args()
args.cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if args.cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=args.batch_size, shuffle=True, **kwargs)


class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-3)


# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        # if batch_idx % args.log_interval == 0:
            # print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            #     epoch, batch_idx * len(data), len(train_loader.dataset),
            #     100. * batch_idx / len(train_loader),
            #     loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(args.batch_size, 1, 28, 28)[:n]])
                # save_image(comparison.cpu(),
                #          'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

if __name__ == "__main__":
    for epoch in range(1, args.epochs + 1):
        train(epoch)
        test(epoch)
        with torch.no_grad():
            sample = torch.randn(args.batch_size, 20).to(device)
            sample = model.decode(sample).cpu()
            save_image(sample.view(args.batch_size, 1, 28, 28),
                       'results/sample_' + str(epoch) + '.png', nrow=10)



====> Epoch: 1 Average loss: 157.5063
====> Test set loss: 123.7445
====> Epoch: 2 Average loss: 119.1971
====> Test set loss: 114.2796
====> Epoch: 3 Average loss: 113.3232
====> Test set loss: 110.8548
====> Epoch: 4 Average loss: 110.6170
====> Test set loss: 108.7808
====> Epoch: 5 Average loss: 109.0432
====> Test set loss: 107.5630
====> Epoch: 6 Average loss: 107.9195
====> Test set loss: 106.8973
====> Epoch: 7 Average loss: 107.2001
====> Test set loss: 106.1299
====> Epoch: 8 Average loss: 106.5994
====> Test set loss: 105.9414
====> Epoch: 9 Average loss: 106.1547
====> Test set loss: 105.4270
====> Epoch: 10 Average loss: 105.7151
====> Test set loss: 104.9749
====> Epoch: 11 Average loss: 105.4201
====> Test set loss: 105.0797
====> Epoch: 12 Average loss: 105.1454
====> Test set loss: 104.9278
====> Epoch: 13 Average loss: 104.8861
====> Test set loss: 104.4679
====> Epoch: 14 Average loss: 104.6485
====> Test set loss: 104.1945
====> Epoch: 15 Average loss: 104.4357
====

In [55]:
sample = torch.randn(args.batch_size, 20).to(device)
sample[:,0] -= 1
sample[:,1] += 1
sample[:,2] -= 1
sample[:,3] += 1
sample[:,4] -= 1
sample[:,5] += 1
sample[:,6] -= 1
sample[:,7] += 1
for ii in range(0, 21):
    sample[:,0] += 0.1
    sample[:,1] -= 0.1
    sample[:,2] += 0.1
    sample[:,3] -= 0.1
    sample[:,4] += 0.1
    sample[:,5] -= 0.1
    sample[:,6] += 0.1
    sample[:,7] -= 0.1
    print(torch.mean(sample))
    images = model.decode(sample).cpu()
    save_image(images.view(args.batch_size, 1, 28, 28)[11,:],
                       'results/pertubation_test_' + str(ii) + '.png', nrow=10)


tensor(0.0001)
tensor(0.0001)
tensor(0.0001)
tensor(0.0001)
tensor(0.0001)
tensor(0.0001)
tensor(0.0001)
tensor(0.0001)
tensor(0.0001)
tensor(0.0001)
tensor(0.0001)
tensor(0.0001)
tensor(0.0001)
tensor(0.0001)
tensor(0.0001)
tensor(0.0001)
tensor(0.0001)
tensor(0.0001)
tensor(0.0001)
tensor(0.0001)
tensor(0.0001)
